In [159]:
import pandas as pd
import numpy as np
import talib

# Data input

In [160]:
# Raw data
BTC = pd.read_csv('../data/bitcoin.csv', index_col='Date', parse_dates=True)
BTC_train = pd.read_csv('../data/bitcoin_train.csv', index_col='Date', parse_dates=True)
BTC_test  = pd.read_csv('../data/bitcoin_test.csv', index_col='Date', parse_dates=True)

ETH = pd.read_csv('../data/ethereum.csv', index_col='Date', parse_dates=True)
ETH_train = pd.read_csv('../data/ethereum_train.csv', index_col='Date', parse_dates=True)
ETH_test  = pd.read_csv('../data/ethereum_test.csv', index_col='Date', parse_dates=True)

# Reconocimiento de patrones

La influencia de los patrones de velas en el comportamiento de la bolsa no tiene una dependencia temporal, es decir, no importa si el patrón aparece antes o después, tan solo que aparezca.

Para cada instante _t_ , acumular el número de patrones en _p_ instantes pasados. Así, dados _P_ patrones diferentes, para cada instante _t_ se crea un array con _P_ elementos en el que _P(i)_ almacena el número de patrones _i_ que se encuentran en esos _p_ instantes pasados.

## Patrones a utilizar

Thomas Bulkowski

Patrones alcistas (Bullish patterns):

1. Three line strike: 3 velas bajistas previas, con recios de close y low muy cercanos. Precio de salida de cuarta vela incluso menor, pero cierra muy alto.
2. Engulfing: Pa
3. Hammer: Sombra entre low y open el doble de grande que el cuerpo.
4. Shooting Star

Patrones de cambio (Reversal patterns):
1. Doji
2. Harami

Patrones bajistas (Bearish patterns):
1. Belt Hold:
2. Evening start: Vela alcista muy grande, vela bajista con nuevo cierre más alto que la anterior y vela bajista con hueco con la segunda.
3. Engulfing

In [174]:
def get_candle_serie(crypto):
    high_values = crypto['High'].values
    low_values = crypto['Low'].values
    open_values = crypto['Open'].values
    close_values = crypto['Close'].values

    bullish_patterns = {}
    bearish_patterns = {}
    reversal_patterns = {}

    # Bullish patterns
    bullish_patterns['threelinestrike'] = np.where(talib.CDL3LINESTRIKE(open_values, high_values, low_values, close_values) == -100)[0]
    bullish_patterns['engulfing'] = np.where(talib.CDLENGULFING(open_values, high_values, low_values, close_values) == 100)[0]
    bullish_patterns['hammer'] = np.where(talib.CDLHAMMER(open_values, high_values, low_values, close_values) == 100)[0]
    bullish_patterns['shootingstar'] = np.where(talib.CDLSHOOTINGSTAR(open_values, high_values, low_values, close_values) == 100)[0]

    # Reversal
    reversal_patterns['doji'] = np.where(talib.CDLDOJI(open_values, high_values, low_values, close_values) == 100)[0]
    reversal_patterns['harami'] = talib.CDLHARAMI(open_values, high_values, low_values, close_values)

    # Bearish patterns
    bearish_patterns['belthold'] = np.where(talib.CDLBELTHOLD(open_values, high_values, low_values, close_values) == 100)[0]
    bearish_patterns['eveningstar'] = np.where(talib.CDLEVENINGSTAR(open_values, high_values, low_values, close_values) == -100)[0]
    bearish_patterns['engulfing'] = np.where(talib.CDLENGULFING(open_values, high_values, low_values, close_values) == -100)[0]

    # Basic candle serie - just bearish and bullish candles are considered
    candle_serie = np.zeros(high_values.shape)
    candle_serie[open_values > close_values] = 0 # Simple bearish candles
    candle_serie[open_values <= close_values] = 1 # Simple bullish candles
    
    # Specific reversal patterns
    candle_serie[reversal_patterns['doji']] = 6
    candle_serie[reversal_patterns['harami']] = 7
    
    # Specific bearish patterns
    candle_serie[bearish_patterns['belthold']] = 8
    candle_serie[bearish_patterns['engulfing']] = 10
    candle_serie[bearish_patterns['eveningstar']] = 9
    
    # Specific bullish patterns
    candle_serie[bullish_patterns['engulfing']] = 3
    candle_serie[bullish_patterns['hammer']] = 4
    candle_serie[bullish_patterns['shootingstar']] = 5
    candle_serie[bullish_patterns['threelinestrike']] = 2

    return candle_serie

In [175]:
# Calculate candle series for both cryptocurr
BTC_candle_serie = get_candle_serie(BTC)
ETH_candle_serie = get_candle_serie(ETH)

## Matriz de entradas

In [176]:
def get_input_cndl_pttrn(serie, p):
    
    # Cast serie to int
    int_serie = serie.astype('int')
    
    # Number of patterns considered
    n_cndl_pttrn = 11
    
    # Window
    begin = 0
    end = p
    
    inputs = np.zeros((len(int_serie)-p, n_cndl_pttrn))
    i=0
    
    
    while end < len(int_serie):
        for value in int_serie[begin:end]:
            inputs[i, value] += 1
        
        i += 1
        begin +=1
        end += 1
        
    return inputs
            

In [177]:
p=15

BTC_candle_inputs = get_input_cndl_pttrn(BTC_candle_serie, p)
ETH_candle_inputs = get_input_cndl_pttrn(ETH_candle_serie, p)

## Guardar matrices

In [199]:
# BTC
np.savetxt('../data/ordinal/BTC_candles_p15.csv', BTC_candle_inputs, fmt='%i')

# ETH
np.savetxt('../data/ordinal/ETH_candles_p15.csv', ETH_candle_inputs, fmt='%i')